In [114]:
#Importing Required Libraries
import folium
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import branca.colormap as cm
%matplotlib inline

In [101]:


#pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

# read taxi data for Jan 2018
data='yellow_tripdata_2018-01.csv'
yt_data = pd.read_csv(data)



,tpep_pickup_datetime,PULocationID,LocationID,Borough
0,2018-01-01 00:21:05,41,41,Manhattan
1,2018-01-01 00:49:32,41,41,Manhattan
2,2018-01-01 00:11:56,41,41,Manhattan
3,2018-01-01 00:58:32,41,41,Manhattan
4,2018-01-01 00:14:09,41,41,Manhattan
5,2018-01-01 00:32:15,41,41,Manhattan
6,2018-01-01 00:38:34,41,41,Manhattan
7,2018-01-01 00:38:54,41,41,Manhattan
8,2018-01-01 00:28:06,41,41,Manhattan
9,2018-01-01 00:36:18,41,41,Manhattan


In [136]:
#Group data by pick up and drop off locations
PUDic=yt_data.groupby("PULocationID")['passenger_count'].sum()
DODic=yt_data.groupby("DOLocationID")['passenger_count'].sum()

In [153]:
#Creating Linear Colormap for pick up and drop off location depending on the min and max values of each group 
linearPU = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=min(PUDic), vmax=max(PUDic),
    caption="Pick Up Pessengers"
)
linearDO = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=min(DODic), vmax=max(DODic),
    caption="Drop off Pessengers"
)


In [210]:
#loading taxi zones data
zones = os.path.join('taxi_zones.json')
geo_json_data = json.load(open(zones))

In [214]:
#merging taxi zones data with the number of pick up and drop off passengers 
for i in geo_json_data["features"]:
    
    i['properties']["PassengersPU"]=int(PUDic[i['properties']['LocationID']] if (i['properties']['LocationID'] in PUDic) else 0)
    i['properties']["PassengersDO"]=int(DODic[i['properties']['LocationID']] if (i['properties']['LocationID'] in DODic) else 0)
    

In [215]:
#drawing map of location with data of pick up passengers
m = folium.Map([40.7831, -73.9712], tiles='cartodbpositron', zoom_start=10)
folium.GeoJson(
    geo_json_data,
    style_function=lambda properties: {
        'fillColor': linearPU(properties['properties']['PassengersPU']),
        #'fillColor': print(properties['properties']['LocationID']),
        'color': 'black',
        'weight': 1,
    },
    tooltip= folium.features.GeoJsonTooltip(fields=['zone','borough','PassengersPU'],aliases=['Zone','Borough','Pick up Passengers'])
).add_to(m)
m.add_child(linearPU)
m.save('PickUpMap.html')

In [216]:
#drawing map of location with data of drop off passengers
m2 = folium.Map([40.7831, -73.9712], tiles='cartodbpositron', zoom_start=10)
folium.GeoJson(
    geo_json_data,
    style_function=lambda properties: {
        'fillColor': linearDO(properties['properties']['PassengersDO']),
        #'fillColor': print(properties['properties']['LocationID']),
        'color': 'black',
        'weight': 1,
    },
    tooltip= folium.features.GeoJsonTooltip(fields=['zone','borough','PassengersDO'],aliases=['Zone','Borough','Drop off Passengers'])
).add_to(m2)
m2.add_child(linearDO)
m2.save('DropOffMap.html')

In [ ]:
#By examination of the map we realize that the zones in the center of manhattan city have hight pickup and drop off numbers
#Zones like Upper east Side North/South Midtown east and midtown center have hightest passenger pickup and drop off numbers
#La Guardia and JFK airport have comparatively much more pick up passengers then drop off passengers